In [75]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pyplot
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical


prep_images normalises the colour values in the image maps to a value between 0 and 1.


In [77]:
def prep_images(train, test):
    train_norm = train.astype('float32')
    test_norm = test.astype('float32')
    train_norm = train_norm / 255.0
    test_norm = test_norm / 255.0
    return train_norm, test_norm


This first model is a simple 1 layer CNN. It can be considered the 'base case.'

In [78]:
def make_model():
    model = Sequential()
    model.add(Conv2D(32,(3,3), activation='relu', padding='same', input_shape=(32, 32, 3)))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    opt = SGD(learning_rate=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

The second model implements the usage of a second hidden layer, batch normalization and the use of dropout on the training data. This model is considered to be far more efficient for that reason.

In [ ]:
def make_second_model():
        model = Sequential()
        model.add(Conv2D(32,(3,3), activation='relu', padding='same', input_shape=(32, 32, 3)))
        model.add(BatchNormalization())
        model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D((2, 2)))
        model.add(Dropout(0.15))
        model.add(Conv2D(32,(3,3), activation='relu', padding='same'))
        model.add(BatchNormalization())
        model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D((2, 2)))
        model.add(Dropout(0.15))
        model.add(Flatten())
        model.add(Dense(64, activation='relu'))
        model.add(Dense(10, activation='softmax'))
        opt = SGD(learning_rate=0.01, momentum=0.9)
        model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
        return model


This function plots the accuracy of classification against training accuracy and validation loss against training loss as a final diagnostic.

In [79]:

def plot_graphs(history):
	pyplot.subplot(1)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	pyplot.subplot(2)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	pyplot.show()


To fun the model.

Epochs- 50. The smaller model was found to converge early so 50 epochs was selected to finish training easier.
Batch Size- 64. Became the most efficient number for this data set via trial and error.

In [80]:
def run_model(images, label_fine, tst_images, tst_fine):
    train, test = prep_images(images,tst_images)
    print(test.shape)
    print(tst_fine.shape)
    model = make_second_model()
    history = model.fit(train, label_fine, epochs=50, batch_size= 64, validation_data=(test, tst_fine), verbose=1)
    _, acc = model.evaluate(test, tst_fine, verbose=0)
    print('> %.3f' % (acc * 100.0))
    plot_graphs(history)


One hot encoding on labels and a transpose to change the dimensions of the image arrays.

In [81]:
images = np.load('src/trnImage.npy')
label_fine = np.load('src/trnLabel_fine.npy')
label_coarse = np.load('src/trnLabel_coarse.npy')
tst_images = np.load('src/tstImage.npy')
tst_fine = np.load('src/tstLabel_fine.npy')
tst_coarse = np.load('src/tstLabel_coarse.npy')


label_coarse = to_categorical(label_coarse)
tst_coarse = to_categorical(tst_coarse)
train = np.transpose(images, [3,0,1,2])
test = np.transpose(tst_images, [3,0,1,2])

run_model(train, label_coarse, test, tst_coarse)




(10000, 32, 32, 3)
(10000, 20)
Epoch 1/50
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
781/782 [============================>.] - ETA: 0s - loss: 3.0019 - accuracy: 0.0478   

KeyboardInterrupt: 